In [198]:
from bs4 import BeautifulSoup
import urllib.request
from queue import Queue

In [199]:
def get_outlinks(url):
    s = set()
    resp = urllib.request.urlopen(url)
    soup = BeautifulSoup(resp, "lxml", from_encoding=resp.info().get_param('charset'))

    for link in soup.find_all('a', href=True):
        outlink = link['href']
        # ignore the links not starting with /wiki/
        if outlink.startswith('/wiki/') and outlink.count('/') == 2 and outlink.find(':') == -1:
            s.add(outlink[len("/wiki/"):])
    return s

In [200]:
def find_path(start_url, end_url_name):
    q = Queue()
    parent = {}
    found = False
    start_url_name = start_url.split('/')[-1]
    parent[start_url_name] = None
    print("Finding path..Please wait")
    
    # get the list of urls on start page
    first_child = get_outlinks(start_url)
    for i in first_child:
        parent[i] = start_url_name
        q.put(i)
        if i == -1:
            found = True
    
    while not q.empty() and not found:
        current_link = q.get()
        child = get_outlinks("https://en.wikipedia.org/wiki/{}".format(current_link))
        for i in child:
            if i not in parent:
                parent[i] = current_link
                q.put(i)
                if i == end_url_name:
                    #we found the link
                    found = True
                    break
                    
    print("The path is being found, creating a path now..")
    #build the path now
    result = []
    temp = end_url_name
    while temp != start_url_name:
        if(temp):
            result.insert(0, parent[temp])
            temp = parent[parent[temp]]
        else:
            break
    result.insert(0, start_url_name) if result[0] != start_url_name else None
    result.append(end_url_name)
    return result

In [205]:
find_path("https://en.wikipedia.org/wiki/United_States", "East_River")

Finding path..Please wait
The path is being found, creating a path now..


['United_States', 'New_York_(state)', 'East_River']